# Modèle ColBERT

On s'intéresse ici au modèle ColBERT, qui propose ...

In [1]:
!pip install --upgrade python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert

  Cloning https://github.com/terrierteam/pyterrier_colbert to c:\users\karna\appdata\local\temp\pip-req-build-jp89wy2z
  Resolved https://github.com/terrierteam/pyterrier_colbert to commit cce6d55a2a869ae354af1930bcd5eff6ced80657
  Cloning https://github.com/cmacdonald/ColBERT.git (to revision v0.2) to c:\users\karna\appdata\local\temp\pip-install-mrj34hd5\colbert_21d2128cda2748c9a92657287d05468a
  Resolved https://github.com/cmacdonald/ColBERT.git to commit f90dc0415655e2c5a0d616100c2e8610a5424686
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached mlflow-1.24.0-py3-none-any.whl (16.5 MB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-win_amd64.whl (1.9 MB)
  Using cached sentencepiece-0.1.96-cp38-cp38-win_amd64.whl (1.1 MB)
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Using cached docker-5.0.3-py2.py3-none-any.whl (146 kB)
  Using cached querystring_parser-1.2.4-py2.py3-none-any.whl (7.9 kB)
  Using cached sqlparse-0.4.2-py3-none-any.whl (42 k

  Running command git clone -q https://github.com/terrierteam/pyterrier_colbert 'C:\Users\karna\AppData\Local\Temp\pip-req-build-jp89wy2z'
  Running command git clone -q https://github.com/cmacdonald/ColBERT.git 'C:\Users\karna\AppData\Local\Temp\pip-install-mrj34hd5\colbert_21d2128cda2748c9a92657287d05468a'
  Running command git checkout -b v0.2 --track origin/v0.2
  Branch 'v0.2' set up to track remote branch 'v0.2' from 'origin'.
  Switched to a new branch 'v0.2'
ERROR: Could not install packages due to an OSError: [WinError 5] Accès refusé: 'C:\\Users\\karna\\anaconda3\\Lib\\site-packages\\pywin32_system32\\pywintypes38.dll'
Consider using the `--user` option or check the permissions.



In [2]:
#initialisation pyterrier
import pyterrier as pt
if not pt.started():
    pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to C:\Users\karna\.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to C:\Users\karna\.pyterrier...
Done


Exception: Unable to find JAVA_HOME

In [ ]:
#récupération du dataset covid19
dataset = pt.datasets.get_dataset('irds:cord19/trec-covid')
topics = dataset.get_topics(variant='description')
qrels = dataset.get_qrels()

#construction de l'index
import os

cord19 = pt.datasets.get_dataset('irds:cord19/trec-covid')
pt_index_path = './terrier_cord19'

if not os.path.exists(pt_index_path + "/data.properties"):
    # create the index, using the IterDictIndexer indexer 
    indexer = pt.index.IterDictIndexer(pt_index_path)

    # we give the dataset get_corpus_iter() directly to the indexer
    # while specifying the fields to index and the metadata to record
    index_ref = indexer.index(cord19.get_corpus_iter(), 
                              fields=('abstract',), 
                              meta=('docno',))

else:
    # if you already have the index, use it.
    index_ref = pt.IndexRef.of(pt_index_path + "/data.properties")

index = pt.IndexFactory.of(index_ref)


import os

if not os.path.exists("terrier_index.zip"):
  !wget http://www.dcs.gla.ac.uk/~craigm/ecir2021-tutorial/terrier_index.zip
  !unzip -j terrier_index.zip -d terrier_index

index_ref = pt.IndexRef.of("./terrier_index/data.properties")
index = pt.IndexFactory.of(index_ref)

## chargement du modèle ColBERT

In [ ]:
import pyterrier_colbert.ranking
colbert_factory = pyterrier_colbert.ranking.ColBERTFactory(
    "http://www.dcs.gla.ac.uk/~craigm/colbert.dnn.zip", None, None)
colbert = colbert_factory.text_scorer(doc_attr='abstract')

Exécution des requêtes du dataset et calcul des métriques

In [ ]:
br = pt.BatchRetrieve(index) % 100
pipeline = br >> pt.text.get_text(dataset, 'abstract') >> colbert
pt.Experiment(
    [br, pipeline],
    topics,
    qrels,
    names=['DPH', 'DPH >> ColBERT'],
    eval_metrics=["map", "ndcg", 'ndcg_cut.10', 'P.10', 'mrt']
)

on souhaite ici visualiser les résultats du modèle, et surtout visualiser l'attention calculée sur le texte.

In [ ]:
res = pipeline(topics.iloc[:1])
res.merge(dataset.get_qrels(), how='left').head()

le premier document est pertinent pour la requête q0. Nous observons ci-dessous la carte d'attention associée au calcul du score : 

In [ ]:
text = dataset.irds_ref().docs_store().get('4dtk1kyh').abstract[:300] + '...' # truncate text
colbert_factory.explain_text('what is the origin of covid 19', text)